# Import Libraries

In [2]:
import os
import shutil
import time
import json
import requests
import tempfile
import numpy as np
import pandas as pd

import boto3
import sagemaker
from datetime import datetime

# Setup Region, session, and role

In [3]:
region = os.environ["AWS_REGION"]
boto_session = boto3.Session(region_name=region)
# sagemaker_boto_client = boto_session.client("sagemaker")


account_id = boto_session.client("sts").get_caller_identity()["Account"]
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)

# Get the AWS EventBridge client
event_bridge_client = boto3.client('events')
lambda_client = boto3.client('lambda')
event_bridge_scheduler = boto3.client("scheduler")

sns_client = boto3.client('sns')
cloudwatch = boto3.client('cloudwatch')

sagemaker_session = sagemaker.Session(
    boto_session=boto_session, sagemaker_client=sm_client
)

# Initialize variables

In [4]:
# S3 prefix
bucket = "artwork-content-trial-bucket"
prefix="mch-artwork-content"
train_data_dir_prefix="data"
pipeline_dir_prefix="pipeline-data"
# training_input_prefix = "content_ovr_anonymised.csv"
training_input_prefix = "ovr_content_data"


# =========================================

# Cloudwatch alarm for sending SNS

In [29]:
def create_pipeline_execution_success_alarm(cloudwatch, alarm_name, namespace, sns_topic_arn, pipeline_name):
    # Create alarm for successful pipeline execution
    exec_alarm_response = cloudwatch.put_metric_alarm(
            AlarmName=alarm_name,
            ComparisonOperator='GreaterThanThreshold',
            EvaluationPeriods=1,
            MetricName='ExecutionSucceeded',
            Namespace=namespace,
            Period=60,
            Statistic='Average',
            Threshold=0,
            ActionsEnabled=True,
            AlarmDescription='Model Retraining and deployment pipeline is successfully executed',
#             AlarmActions=[
#                 sns_topic_arn
#             ],
            Dimensions=[
                {
                  'Name': 'PipelineName',
                  'Value': pipeline_name
                }]
        )


In [30]:
def create_model_reg_success_alarm(cloudwatch, alarm_name, namespace, sns_topic_arn, pipeline_name, step_name):
    
    # Create alarm for successful model registration
    reg_alarm_response = cloudwatch.put_metric_alarm(
        AlarmName= alarm_name,
        ComparisonOperator='GreaterThanThreshold',
        EvaluationPeriods=1,
        MetricName='StepSucceeded',
        Namespace=namespace,
        Period=60,
        Statistic='Minimum',
        Threshold=0,
        ActionsEnabled=True,
        AlarmDescription='Model Version registered successfully',
#         AlarmActions=[
#           sns_topic_arn
#         ],
        Dimensions=[
            {
              'Name': 'PipelineName',
              'Value': pipeline_name
            },
            {
              'Name': 'StepName',
              'Value': step_name
            },
        ]
    )
             

In [12]:
def sns_subscribe(sns_client, sns_topic_arn, email_address):
    # subcribe to emails
#     for email_address in email_addresses:
    subscribe_response = sns_client.subscribe(
                                                TopicArn= sns_topic_arn,
                                                Protocol='email',
                                            Endpoint=email_address,
                                                ReturnSubscriptionArn=True
                                            )
    
    return subscribe_response

In [44]:
def set_sns_alarm(sns_client, cloudwatch,  model_alarm_name, pipeline_alarm_name, topic_name, sns_topic_arn, email_address, namespace, pipeline_name, step_name):
    
    topic_arns = sns_client.list_topics()['Topics']
    
    for topic in topic_arns:
        if topic_name in topic['TopicArn']:
            print("Topic name {} found in {}".format(topic_name, topic['TopicArn']))
            break
    else:
        print('new topic created')
        topic_response = sns_client.create_topic(
                            Name=topic_name
                        )
    # print(topic_response)
    
    # sns_subscribe(sns_client, sns_topic_arn, email_address)
    
    create_pipeline_execution_success_alarm(cloudwatch, pipeline_alarm_name, namespace, sns_topic_arn, pipeline_name)
    
    create_model_reg_success_alarm(cloudwatch, model_alarm_name, namespace, sns_topic_arn, pipeline_name, step_name)
    

In [45]:
namespace = 'AWS/Sagemaker/ModelBuildingPipeline'
pipeline_name ='artwork-content-pipeline-demo'
topic_name = 'PipelineExecutionAlarmTopic'
sns_topic_arn = f'arn:aws:sns:{region}:{account_id}:{topic_name}'
step_name = 'RegisterModelStep-RegisterModel'
email_address='vchitrakathi@dminc.com'#, 'chitrakathi03@gmail.com']
model_alarm_name='Model_Version_Registration_Alarm'
pipeline_alarm_name='SageMaker_Pipeline_Execution_Alarm'

In [46]:
set_sns_alarm(sns_client, cloudwatch, model_alarm_name, pipeline_alarm_name, topic_name, sns_topic_arn, email_address, namespace, pipeline_name, step_name)

Topic name PipelineExecutionAlarmTopic found in arn:aws:sns:us-east-1:791574662255:PipelineExecutionAlarmTopic


# ===============================================

# Delete Alarms

In [28]:
# cloudwatch.delete_alarms(AlarmNames=[model_alarm_name, pipeline_alarm_name])